In [1]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pyreadstat as pyr
import json
import numpy as np
from spss_import import read_sav
from lxml import etree
from xml_functions import remove_empty_elements, add_cdi_element, add_identifier, add_ddiref
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [2]:
#help(pyr.read_sav)

In [3]:
spssfile = f"files/SPSS_Example2.sav"
df, df_meta = read_sav(spssfile)
df.head()

,RID,MARST,PWT,testvar
0,10000001,3,537,1
1,10000002,1,231,1
2,10000003,7,599,1
3,10000004,8,4003,7
4,10000005,9,598,8


In [4]:
Label = df_meta.column_names_to_labels
Values = df_meta.variable_value_labels
Missing = df_meta.missing_ranges
Format = df_meta.original_variable_types
Measure = df_meta.variable_measure

In [5]:
# DataStore
def generate_DataStore(df_meta):
    element = add_cdi_element(root, 'DataStore')
    add_cdi_element(element, 'allowsDuplicates', "false")
    add_identifier(element, f"#dataStore")
    add_cdi_element(element, 'recordCount', df_meta.number_rows)
    LogicalRecord = add_cdi_element(element, 'DataStore_has_LogicalRecord')
    add_ddiref(LogicalRecord, f"#logicalRecord", agency, "LogicalRecord")

In [6]:
# logicalRecord
def generate_LogicalRecord(df_meta):
    element = add_cdi_element(root, 'LogicalRecord')
    add_identifier(element, f"#logicalRecord")
    LogicalRecord_organizes_DataSet = add_cdi_element(element, 'LogicalRecord_organizes_DataSet')
    add_ddiref(LogicalRecord_organizes_DataSet, f"#wideDataSet", agency, "WideDataSet")
    for idx, variable in enumerate(df_meta.column_names):
        LogicalRecord_has_InstanceVariable = add_cdi_element(element, 'LogicalRecord_has_InstanceVariable')
        add_ddiref(LogicalRecord_has_InstanceVariable, f"#instanceVariable-{variable}", agency, "InstanceVariable")

In [7]:
# WideDataSet
def generate_WideDataSet(df_meta):       
    element = add_cdi_element(root, 'WideDataSet')
    add_identifier(element, f"#wideDataSet")
    DataSet_isStructuredBy_DataStructure = add_cdi_element(element, 'DataSet_isStructuredBy_DataStructure')
    add_ddiref(DataSet_isStructuredBy_DataStructure, f"#wideDataStructure", agency, "WideDataStructure")

In [8]:
# WideDataStructure
def generate_WideDataStructure(df_meta):
    element = add_cdi_element(root, 'WideDataStructure')
    add_identifier(element, f"#wideDataStructure")
    
    DataStructure_has_DataStructureComponent = add_cdi_element(element, 'DataStructure_has_DataStructureComponent')
    add_ddiref(DataStructure_has_DataStructureComponent, f"#identifierComponent-{df_meta.column_names[0]}", agency, "IdentifierComponent")
    
    for x, variable in enumerate(df_meta.column_names[1:]): 
        DataStructure_has_DataStructureComponent = add_cdi_element(element, 'DataStructure_has_DataStructureComponent')
        add_ddiref(DataStructure_has_DataStructureComponent, f"#measureComponent-{variable}", agency, "MeasureComponent")

    DataStructure_has_PrimaryKey = add_cdi_element(element, 'DataStructure_has_PrimaryKey')
    add_ddiref(DataStructure_has_PrimaryKey, f"#primaryKey", agency, "PrimaryKey")

In [9]:
# IdentifierComponent
def generate_IdentifierComponent(df_meta):
    element = add_cdi_element(root, 'IdentifierComponent')
    add_identifier(element, f"#identifierComponent-{df_meta.column_names[0]}")
    DataStructureComponent_isDefinedBy_RepresentedVariable = add_cdi_element(element, 'DataStructureComponent_isDefinedBy_RepresentedVariable')
    add_ddiref(DataStructureComponent_isDefinedBy_RepresentedVariable, f"#instanceVariable-{df_meta.column_names[0]}", agency, "InstanceVariable")

In [10]:
# MeasureComponent
def generate_MeasureComponent(df_meta):
    for x, variable in enumerate(df_meta.column_names[1:]): 
        MeasureComponent = add_cdi_element(root, 'MeasureComponent')
        add_identifier(MeasureComponent, f"#measureComponent-{variable}")
        DataStructureComponent_isDefinedBy_RepresentedVariable = add_cdi_element(MeasureComponent, 'DataStructureComponent_isDefinedBy_RepresentedVariable')
        add_ddiref(DataStructureComponent_isDefinedBy_RepresentedVariable, f"#instanceVariable-{df_meta.column_names[0]}", agency, "InstanceVariable")

In [11]:
# PrimaryKey
def generate_PrimaryKey(df_meta):
    element = add_cdi_element(root, 'PrimaryKey')
    add_identifier(element, f"#primaryKey")
    PrimaryKey_isComposedOf_PrimaryKeyComponent = add_cdi_element(element, 'PrimaryKey_isComposedOf_PrimaryKeyComponent')
    add_ddiref(PrimaryKey_isComposedOf_PrimaryKeyComponent, f"#primaryKeyComponent", agency, "PrimaryKeyComponent")

In [12]:
# PrimaryKeyComponent
def generate_PrimaryKeyComponent(df_meta):
    element = add_cdi_element(root, 'PrimaryKeyComponent')
    add_identifier(element, f"#primaryKeyComponent")
    PrimaryKeyComponent_correspondsTo_DataStructureComponent = add_cdi_element(element, 'PrimaryKeyComponent_correspondsTo_DataStructureComponent')
    add_ddiref(PrimaryKeyComponent_correspondsTo_DataStructureComponent, f"#identifierComponent-{df_meta.column_names[0]}", agency, "IdentifierComponent")

In [13]:
def generate_InstanceVariable(df_meta):
    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        element = add_cdi_element(root, 'InstanceVariable')
        displayLabel = add_cdi_element(element, 'displayLabel')
        languageSpecificString = add_cdi_element(displayLabel, 'languageSpecificString')
        add_cdi_element(languageSpecificString, 'content', f"{df_meta.column_labels[idx]}")
        add_identifier(element, f"#instanceVariable-{variable}")
        name = add_cdi_element(element, 'name')
        add_cdi_element(name, 'name', f"{variable}")
        hasIntendedDataType = add_cdi_element(element, 'hasIntendedDataType')
        add_cdi_element(hasIntendedDataType, 'name', f"{df_meta.original_variable_types[variable]}")

        # Check if variable has sentinel concepts
        if variable in df_meta.missing_ranges or (len(df_meta.missing_ranges) == 0 and variable in df_meta.missing_user_values):
            RepresentedVariable_takesSentinelValuesFrom_SentinelValueDomain = add_cdi_element(element, 'RepresentedVariable_takesSentinelValuesFrom_SentinelValueDomain')
            add_ddiref(RepresentedVariable_takesSentinelValuesFrom_SentinelValueDomain, f"#sentinelValueDomain-{variable}", agency, "SentinelValueDomain")
        RepresentedVariable_takesSubstantiveValuesFrom_SubstantiveValueDomain = add_cdi_element(element, 'RepresentedVariable_takesSubstantiveValuesFrom_SubstantiveValueDomain')
        add_ddiref(RepresentedVariable_takesSubstantiveValuesFrom_SubstantiveValueDomain, f"#substantiveValueDomain-{variable}", agency, 'SubstantiveValueDomain')

In [14]:
# SubstantiveValueDomain
def generate_SubstantiveValueDomain(df_meta):
    for var in df_meta.column_names:
        element = add_cdi_element(root, 'SubstantiveValueDomain')
        add_identifier(element, f"#substantiveValueDomain-{var}")
        if var in df_meta.variable_value_labels:
            SubstantiveValueDomain_takesValuesFrom_EnumerationDomain = add_cdi_element(element, 'SubstantiveValueDomain_takesValuesFrom_EnumerationDomain')
            add_ddiref(SubstantiveValueDomain_takesValuesFrom_EnumerationDomain, f"#substantiveCodelist-{var}", agency, 'CodeList')
        SubstantiveValueDomain_isDescribedBy_ValueAndConceptDescription = add_cdi_element(element, 'SubstantiveValueDomain_isDescribedBy_ValueAndConceptDescription')
        add_ddiref(SubstantiveValueDomain_isDescribedBy_ValueAndConceptDescription, f"#substantiveValueAndConceptDescription-{var}", agency, "ValueAndConceptDescription")


In [15]:
# SentinelConceptualDomain
def generate_SentinelValueDomain(df_meta):
    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    for variable in relevant_variables:
        element = add_cdi_element(root, 'SentinelValueDomain')
        add_identifier(element, f"#sentinelValueDomain-{variable}")
        
        if variable in df_meta.variable_value_labels:
            SentinelValueDomain_takesValuesFrom_EnumerationDomain = add_cdi_element(element, 'SentinelValueDomain_takesValuesFrom_EnumerationDomain')
            add_ddiref(SentinelValueDomain_takesValuesFrom_EnumerationDomain, f"#sentinelCodelist-{variable}", agency, 'CodeList')
        SentinelValueDomain_isDescribedBy_ValueAndConceptDescription = add_cdi_element(element, 'SentinelValueDomain_isDescribedBy_ValueAndConceptDescription')
        add_ddiref(SentinelValueDomain_isDescribedBy_ValueAndConceptDescription, f"#sentinelValueAndConceptDescription-{variable}", agency, "ValueAndConceptDescription")


In [16]:

# ValueAndConceptDescription
def generate_ValueAndConceptDescription(df_meta):

    
    # Determine the relevant variables based on the presence of missing values
    relevant_variables = {}
    if df_meta.missing_ranges:
        relevant_variables = df_meta.missing_ranges
    elif df_meta.missing_user_values:
        relevant_variables = df_meta.missing_user_values

    json_ld_data = []

    # recode classification level
    class_level = {'nominal': 'Nominal', 'scale': 'Continuous', 'ordinal': 'Ordinal'}
    for variable in df_meta.column_names:
        element = add_cdi_element(root, 'ValueAndConceptDescription')
        add_cdi_element(element, 'classificationLevel', f"{class_level[df_meta.variable_measure[variable]]}")
        add_identifier(element, f"#substantiveValueAndConceptDescription-{variable}")

        # Add sentinelValueAndConceptDescription only if the condition is met
        if variable in relevant_variables:
            values = relevant_variables[variable]
            if isinstance(values[0], dict):  # Check if the values are dictionaries
                all_lo_values = [d['lo'] for d in values]
                all_hi_values = [d['hi'] for d in values]
                min_val = min(all_lo_values)
                max_val = max(all_hi_values)
            else:
                min_val, max_val = min(values), max(values)
        
            element = add_cdi_element(root, 'ValueAndConceptDescription')
            description = add_cdi_element(element, 'description')
            languageSpecificString = add_cdi_element(description, 'languageSpecificString')
            add_cdi_element(languageSpecificString, 'content', str(values))
            add_identifier(element, f"#sentinelValueAndConceptDescription-{variable}")
            add_cdi_element(element, "maximumValueExclusive", str(max_val))
            add_cdi_element(element, "minimumValueExclusive", str(min_val))

            json_ld_data.append({
                "@id": f"#sentinelValueAndConceptDescription-{variable}",
                "@type": "ValueAndConceptDescription",
                "description": str(values),
                "minimumValueExclusive": str(min_val),
                "maximumValueExclusive": str(max_val),
            })


In [17]:
# SubstantiveCodeList
def generate_CodeList(df_meta):
    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        element = add_cdi_element(root, 'CodeList')
        add_identifier(element, f"#substantiveCodelist-{variable_name}")

        name = add_cdi_element(element, 'name')
        add_cdi_element(name, 'name', f"#substantiveCodelist-{variable_name}")
        
        add_cdi_element(element, 'allowsDuplicates', "false")

        excluded_values = set()

        # Check if variable_name is in relevant_variables
        if variable_name in relevant_variables:

            # If the relevant variable data is based on ranges and contains dictionaries
            if isinstance(relevant_variables[variable_name], list) and all(
                    isinstance(item, dict) for item in relevant_variables[variable_name]):
                for dict_range in relevant_variables[variable_name]:
                    lo_is_numeric = isinstance(dict_range['lo'], (int, float)) or (
                            isinstance(dict_range['lo'], str) and dict_range['lo'].isnumeric()
                    )
                    hi_is_numeric = isinstance(dict_range['hi'], (int, float)) or (
                            isinstance(dict_range['hi'], str) and dict_range['hi'].isnumeric()
                    )

                    if lo_is_numeric and hi_is_numeric:
                        excluded_values.update(
                            range(int(float(dict_range['lo'])), int(float(dict_range['hi'])) + 1)
                        )
                    elif isinstance(dict_range['lo'], str):
                        excluded_values.add(dict_range['lo'])
                    else:
                        print(f"Warning: Unsupported 'lo' value: {dict_range['lo']}")

            # If the relevant variable data contains strings (user-defined missing values)
            elif isinstance(relevant_variables[variable_name], list):
                excluded_values.update(set(map(str, relevant_variables[variable_name])))

        for value in values_dict.keys():
            excluded_values_str = {str(i) for i in excluded_values}
            if (not value in excluded_values) and (not str(value) in excluded_values_str) :
                CodeList_has_Code = add_cdi_element(element, 'CodeList_has_Code')
                add_ddiref(CodeList_has_Code, f"#code-{value}-{variable_name}", agency, "Code")

In [18]:
# SentinelCodelist
def generate_SentinelCodelist(df_meta):
    json_ld_data = []

    # Determine the relevant variables based on the presence of missing values
    relevant_variables = df_meta.missing_ranges if len(df_meta.missing_ranges) > 0 else df_meta.missing_user_values

    def is_value_in_range(value, ranges):
        """Check if a value is in any of the given ranges."""
        for range_dict in ranges:
            if range_dict['lo'] <= value <= range_dict['hi']:
                return True
        return False

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        element = add_cdi_element(root, 'CodeList')
        add_identifier(element, f"#sentinelCodelist-{variable_name}")

        name = add_cdi_element(element, 'name')
        add_cdi_element(name, 'name', f"#sentinelCodelist-{variable_name}")
        
        add_cdi_element(element, 'allowsDuplicates', "false")
            
        if variable_name in relevant_variables:
            if variable_name in df_meta.missing_ranges:
                # Use a for loop to generate the hasTopConcept list
                for value in values_dict.keys():
                    if is_value_in_range(value, df_meta.missing_ranges[variable_name]):
                        CodeList_has_Code = add_cdi_element(element, 'CodeList_has_Code')
                        add_ddiref(CodeList_has_Code, f"#code-{value}-{variable_name}", agency, "Code")
            else:
                excluded_values = set(df_meta.missing_user_values[variable_name])
                for value in values_dict.keys():
                    if value in excluded_values:
                        CodeList_has_Code = add_cdi_element(element, 'CodeList_has_Code')
                        add_ddiref(CodeList_has_Code, f"#code-{value}-{variable_name}", agency, "Code")


In [19]:
# Code
def generate_Code(df_meta):
    for variable_name, values_dict in df_meta.variable_value_labels.items():
        for key, value in values_dict.items():
            element = add_cdi_element(root, 'Code')
            add_identifier(element, f"#code-{key}-{variable_name}")
            Code_denotes_Category = add_cdi_element(element, 'Code_denotes_Category')
            add_ddiref(Code_denotes_Category, f"#category-{value}", agency, "Category")
            Code_uses_Notation = add_cdi_element(element, 'Code_uses_Notation')
            add_ddiref(Code_uses_Notation, f"#notation-{key}", agency, "Notation")

In [20]:
# Category and Notation
def generate_Category_Notation(df_meta):
    notations = list(set(key for values_dict in df_meta.variable_value_labels.values() for key in values_dict.keys()))
    cats = list(set(value for values_dict in df_meta.variable_value_labels.values() for value in values_dict.values()))
    
    for cat in cats:
        element = add_cdi_element(root, 'Category')
        displayLabel = add_cdi_element(element, 'displayLabel')
        languageSpecificString = add_cdi_element(displayLabel, 'languageSpecificString')
        add_cdi_element(languageSpecificString, 'content', f"{cat}")
        add_identifier(element, f"#category-{cat}")
        name = add_cdi_element(element, 'name')
        add_cdi_element(name, 'name', f"{cat}")

    for note in notations:
        element = add_cdi_element(root, 'Notation')
        content = add_cdi_element(element, 'content')
        add_cdi_element(content, 'content', f"{note}")
        add_identifier(element, f"#notation-{note}")

In [21]:
# PhysicalDataset
def generate_PhysicalDataset(df_meta, spssfile):
    element = add_cdi_element(root, 'PhysicalDataSet')
    add_cdi_element(element, 'allowsDuplicates', "false")
    add_identifier(element, f"#physicalDataset")
    add_cdi_element(element, 'physicalFileName', spssfile)    
    PhysicalDataSet_formats_DataStore = add_cdi_element(element, f"PhysicalDataSet_formats_DataStore")
    add_ddiref(PhysicalDataSet_formats_DataStore, f"#dataStore", agency, "DataStore")
    PhysicalDataSet_has_PhysicalRecordSegment = add_cdi_element(element, 'PhysicalDataSet_has_PhysicalRecordSegment')
    add_ddiref(PhysicalDataSet_has_PhysicalRecordSegment, f"#physicalRecordSegment", agency, "PhysicalRecordSegment")

In [22]:
# PhysicalRecordSegment
def generate_PhysicalRecordSegment(df_meta):
    element = add_cdi_element(root, 'PhysicalRecordSegment')
    add_cdi_element(element, 'allowsDuplicates', "false")
    add_identifier(element, f"#physicalRecordSegment")
   
    PhysicalRecordSegment_has_PhysicalSegmentLayout = add_cdi_element(element, f"PhysicalRecordSegment_has_PhysicalSegmentLayout")
    add_ddiref(PhysicalRecordSegment_has_PhysicalSegmentLayout, f"#physicalSegmentLayout", agency, "PhysicalSegmentLayout")
    PhysicalRecordSegment_mapsTo_LogicalRecord = add_cdi_element(element, f"PhysicalRecordSegment_mapsTo_LogicalRecord")
    add_ddiref(PhysicalRecordSegment_mapsTo_LogicalRecord, f"#logicalRecord", agency, "LogicalRecord")

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):


        for i, x in enumerate(df[variable]):
            PhysicalRecordSegment_has_DataPointPosition = add_cdi_element(element, f"PhysicalRecordSegment_has_DataPointPosition")
            add_ddiref(PhysicalRecordSegment_has_DataPointPosition, f"#dataPointPosition-{i}-{variable}", agency, "DataPointPosition")    

In [23]:
# PhysicalSegmentLayout
def generate_PhysicalSegmentLayout(df_meta):
    element = add_cdi_element(root, 'PhysicalSegmentLayout')
    add_cdi_element(element, 'allowsDuplicates', "false")
    add_identifier(element, f"#physicalSegmentLayout")
    add_cdi_element(element, 'isDelimited', "false")
    add_cdi_element(element, 'isFixedWidth', "false")

    for x, variable in enumerate(df_meta.column_names):
        PhysicalSegmentLayout_has_ValueMapping = add_cdi_element(element, f"PhysicalSegmentLayout_has_ValueMapping")
        add_ddiref(PhysicalSegmentLayout_has_ValueMapping, f"#valueMapping-{variable}", agency, "ValueMapping")

    for x, variable in enumerate(df_meta.column_names):
        PhysicalSegmentLayout_has_ValueMappingPosition = add_cdi_element(element, f"PhysicalSegmentLayout_has_ValueMappingPosition")
        add_ddiref(PhysicalSegmentLayout_has_ValueMappingPosition, f"#valueMappingPosition-{variable}", agency, "ValueMappingPosition")

In [24]:
# ValueMapping
def generate_ValueMapping(df, df_meta):
    
    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        element = add_cdi_element(root, 'ValueMapping')
        add_cdi_element(element, 'defaultValue', "")
        add_identifier(element, f"#valueMapping-{variable}")

        for i, x in enumerate(df[variable]):
            ValueMapping_formats_DataPoint = add_cdi_element(element, f"ValueMapping_formats_DataPoint")
            add_ddiref(ValueMapping_formats_DataPoint, f"#dataPoint-{i}-{variable}", agency, "DataPoint")            

In [25]:
# ValueMappingPosition
def generate_ValueMappingPosition(df_meta):

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        element = add_cdi_element(root, 'ValueMappingPosition')
        add_identifier(element, f"#valueMappingPosition-{variable}")
        add_cdi_element(element, 'value', idx)
        ValueMappingPosition_indexes_ValueMapping = add_cdi_element(element, 'ValueMappingPosition_indexes_ValueMapping')
        add_ddiref(ValueMappingPosition_indexes_ValueMapping, f"#valueMapping-{variable}", agency, "ValueMapping") 

In [26]:
# DataPoint
def generate_DataPoint(df, df_meta):

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            element = add_cdi_element(root, 'DataPoint')
            add_identifier(element, f"#dataPoint-{idx}-{variable}")
            DataPoint_isDescribedBy_InstanceVariable = add_cdi_element(element, 'DataPoint_isDescribedBy_InstanceVariable')
            add_ddiref(DataPoint_isDescribedBy_InstanceVariable, f"#instanceVariable-{variable}", agency, "InstanceVariable") 

In [27]:
# DataPointPosition
def generate_DataPointPosition(df, df_meta):

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            element = add_cdi_element(root, 'DataPointPosition')
            add_identifier(element, f"#dataPointPosition-{idx}-{variable}")
            add_cdi_element(element, 'value', idx)
            DataPointPosition_indexes_DataPoint = add_cdi_element(element, 'DataPointPosition_indexes_DataPoint')
            add_ddiref(DataPointPosition_indexes_DataPoint, f"#dataPoint-{idx}-{variable}", agency, "DataPoint") 

In [28]:
def generate_InstanceValue(df, df_meta):    

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            element = add_cdi_element(root, 'InstanceValue')
            content = add_cdi_element(element, "content")
            add_cdi_element(content, "content", value)
            add_identifier(element, f"#instanceValue-{idx}-{variable}")

            if variable in df_meta.missing_ranges:
                for range_dict in df_meta.missing_ranges[variable]:
                    if isinstance(range_dict['lo'], float):
                        # convert value to float for comparison
                        value = float(value)
                    if range_dict['lo'] <= value <= range_dict['hi'] and isinstance(value, (str, int, float)):
                        InstanceValue_hasValueFrom_ValueDomain = add_cdi_element(element, 'InstanceValue_hasValueFrom_ValueDomain')
                        add_ddiref(InstanceValue_hasValueFrom_ValueDomain, f"#sentinelValueDomain-{variable}", agency, "SentinelValueDomain")
                        break
                else:
                    InstanceValue_hasValueFrom_ValueDomain = add_cdi_element(element, 'InstanceValue_hasValueFrom_ValueDomain')
                    add_ddiref(InstanceValue_hasValueFrom_ValueDomain, f"#substantiveValueDomain-{variable}", agency, "SubstantiveValueDomain")

            else:
                InstanceValue_hasValueFrom_ValueDomain = add_cdi_element(element, 'InstanceValue_hasValueFrom_ValueDomain')
                add_ddiref(InstanceValue_hasValueFrom_ValueDomain, f"#substantiveValueDomain-{variable}", agency, "SubstantiveValueDomain")

            InstanceValue_isStoredIn_DataPoint = add_cdi_element(element, 'InstanceValue_isStoredIn_DataPoint')
            add_ddiref(InstanceValue_isStoredIn_DataPoint, f"#dataPoint-{idx}-{variable}", agency, "DataPoint")

In [29]:
# Define the namespace
nsmap = {'cdi': 'http://ddialliance.org/Specification/DDI-CDI/1.0/XMLSchema/'}
# Create the root element
root = etree.Element(etree.QName(nsmap['cdi'], 'DDICDIModels'), nsmap=nsmap)
root.set('{http://www.w3.org/2001/XMLSchema-instance}schemaLocation',
         'http://ddialliance.org/Specification/DDI-CDI/1.0/XMLSchema/ https://ddi-cdi-resources.bitbucket.io/2023-11-12/encoding/xml-schema/ddi-cdi.xsd')
agency='int.esseric'

In [30]:
generate_DataStore(df_meta)
generate_LogicalRecord(df_meta)
generate_WideDataSet(df_meta)
generate_WideDataStructure(df_meta)
generate_IdentifierComponent(df_meta)
generate_MeasureComponent(df_meta)
generate_PrimaryKey(df_meta)
generate_PrimaryKeyComponent(df_meta)
generate_InstanceVariable(df_meta)
generate_SubstantiveValueDomain(df_meta)
generate_SentinelValueDomain(df_meta)
generate_ValueAndConceptDescription(df_meta)
generate_CodeList(df_meta)
generate_SentinelCodelist(df_meta)
generate_Code(df_meta)
generate_Category_Notation(df_meta)
generate_PhysicalDataset(df_meta, spssfile)
generate_PhysicalRecordSegment(df_meta)
generate_PhysicalSegmentLayout(df_meta)
generate_ValueMapping(df, df_meta)
generate_ValueMappingPosition(df_meta)
generate_DataPoint(df, df_meta)
generate_DataPointPosition(df, df_meta)
generate_InstanceValue(df, df_meta)

In [31]:
# Add XML declaration and write XML file
xml_string = etree.tostring(root, encoding='UTF-8', xml_declaration=True, pretty_print=True)

# Add the comment as the second line
xml_string_with_comment = xml_string.replace(b'?>', b'?>\n<!-- CDI, version 1, 2024.01.20 -->', 1)

with open(r'files/CDI.xml', 'wb') as f:
    f.write(xml_string_with_comment)

##### 